### In this project we will use Amazon Comprehend to classify reviews as positive, negative, neutral or mixed

In [4]:
import boto3
client = boto3.client('s3')
path = 's3://mcit-aws/1429_1.csv'

In [6]:
import pandas as pd
df = pd.read_csv(path)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

Now we will manipulate the dataframe to have only 1000 review and we will need only reviews.rating and reviews.text

In [18]:
df=df[["reviews.rating", "reviews.text"]]
df=df.head(1000)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviews.rating  1000 non-null   float64
 1   reviews.text    1000 non-null   object 
dtypes: float64(1), object(1)
memory usage: 15.8+ KB


**Let's connect to Amazon Comprehend**

In [21]:
comprehend = boto3.client('comprehend', region_name='us-east-1')

First let's test with a sample sentenece and see Amazon Comprehend's sentiment output

In [23]:
sample_text="It’s always a great day when I can randomly put my equestrian knowledge to good use at work! #AWS #BePeculiar" 
comprehend.detect_sentiment(Text=sample_text, LanguageCode='en')

{'Sentiment': 'POSITIVE',
 'SentimentScore': {'Positive': 0.9705585241317749,
  'Negative': 0.0006954022101126611,
  'Neutral': 0.028633279725909233,
  'Mixed': 0.00011291868577245623},
 'ResponseMetadata': {'RequestId': '06026413-8364-4236-b54b-1c1eb738fa6d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '06026413-8364-4236-b54b-1c1eb738fa6d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '166',
   'date': 'Fri, 27 Nov 2020 15:52:49 GMT'},
  'RetryAttempts': 0}}

In [28]:
sentiment.get('Sentiment')

'POSITIVE'

**Our next step to use Amazon Comprehend to output sentmient to each of the 1000 reviews in our dataframe**

In [62]:
#We will save the sentiments in sentmients list for the 1000 reviews
sentiments = []
for i in range(len(df)):
    sentiments.append(comprehend.detect_sentiment(Text=df['reviews.text'][i], LanguageCode='en').get('Sentiment'))
    

In [65]:
#adding a new column in the dataframe for the sentiment of each review
df['sentiment']=sentiments

In [70]:
pd.options.display.max_colwidth = 500
df.head(50)

,reviews.rating,reviews.text,sentiment
0,5.0,This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.,POSITIVE
1,5.0,great for beginner or experienced person. Bought as a gift and she loves it,POSITIVE
2,5.0,"Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already...",POSITIVE
3,4.0,I've had my Fire HD 8 two weeks now and I love it. This tablet is a great value.We are Prime Members and that is where this tablet SHINES. I love being able to easily access all of the Prime content as well as movies you can download and watch laterThis has a 1280/800 screen which has some really nice look to it its nice and crisp and very bright infact it is brighter then the ipad pro costing $900 base model. The build on this fire is INSANELY AWESOME running at only 7.7mm thick and the smo...,POSITIVE
4,5.0,"I bought this for my grand daughter when she comes over to visit. I set it up with her as the user, entered her age and name and now Amazon makes sure that she only accesses sites and content that are appropriate to her age. Simple to do and she loves the capabilities. I also bought and installed a 64gig SD card which gives this little tablet plenty of storage. For the price I think this tablet is best one out there. You can spend hundreds of dollars more for additional speed and capacity bu...",POSITIVE
5,5.0,This amazon fire 8 inch tablet is the perfect size. I purchased it for my husband so that he has a bigger screen than just his phone. He had gotten me one a few years ago so I knew it would be a good purchase.,POSITIVE
6,4.0,"Great for e-reading on the go, nice and light weight, and for the price point given, definitely worth the purchase.",POSITIVE
7,5.0,"I gave this as a Christmas gift to my inlaws, husband and uncle. They loved it and how easy they are to use with fantastic features!",POSITIVE
8,5.0,"Great as a device to read books. I like that it links with my borrowed library e-books. Switched from another popular tablet brand and I am happy with the choice I made. It took some time to get books from my previous non-Kindle reader, but finally figured out a way!",POSITIVE
9,5.0,I love ordering books and reading them with the reader.,POSITIVE


In [68]:
df['sentiment'].value_counts()

POSITIVE    835
MIXED       113
NEGATIVE     44
NEUTRAL       8
Name: sentiment, dtype: int64

From the above there are 8 reviews classfied as neutral so let's examine them as an example

In [71]:
df.loc[df['sentiment'] == 'NEUTRAL']

,reviews.rating,reviews.text,sentiment
42,4.0,Does all basic functions of a tablet. Hardcore users may want an iPad.,NEUTRAL
68,4.0,Bought as a Christmas gift for someone. I bought iPad as well. Will update when it's used.,NEUTRAL
147,5.0,This Tablet was purchased for our Grandmother so she could play games and read books on.,NEUTRAL
247,5.0,I can do a lot of things on my Kindle I used to rely on the computer for.,NEUTRAL
451,4.0,Have not used these yet as they are Christmas gifts,NEUTRAL
805,5.0,Bought this for my niece for Christmas. She mostly uses as ereader and checking email.,NEUTRAL
881,5.0,"Bought on Black Friday for $60, add a trifold case, portable TV while I do dishes/cook/anything",NEUTRAL
957,5.0,"I have used the Kindle mostly for reading. I have downloaded books from Amazon, some free some purchased, that I have heard about for years and now will read. I have also read Manga from the Internet. My use of the internet has also included looking up gardening hints, animal care, and dog breed characteristics.",NEUTRAL


From the above we can see that the classification works perfectly noting that comparing the rating with the output sentiment and getting confusion matrix will not be very useful as we can see that the output of comprehend has 4 values(postivie, negative, neutral and mixed) while the data labels are values from 1 to 5